In [24]:
#imports
import requests
import pandas as pd
import urllib.request
import lxml.html as lh

In [25]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)
records = doc.xpath('//tr')

In [34]:
#get headers and table rows
i=0
#list for records
codeList = []
#get column headers
for t in records[0]:
    i+=1
    colName = t.text_content().strip('\n')
    codeList.append((colName,[]))

In [37]:
#reset index, get record data
#get remaining rows
for j in range(1, len(records)):
    r=records[j]
    if len(r) != 3:
        break
    i=0
    for t in r.iterchildren():
        data = t.text_content().strip('\n')
        codeList[i][1].append(data)
        i+=1

In [40]:
#convert the list from above into a dataframe
Dict = {title:column for (title,column) in codeList}
df = pd.DataFrame(Dict)
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [49]:
3filter records
df_filtered = df[df['Borough']!='Not assigned']
df_filtered.shape

(104, 3)

<H1>Add latitude and longitude to the postal code data collected above.</H1>

In [55]:
#load lat long data
df_latlon = pd.read_csv(r'http://cocl.us/Geospatial_data')
#add columns to prior dataframe
df_full = df_filtered.join(df_latlon.set_index('Postal Code'),on='Postal Code')
df_full.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
